<a href="https://colab.research.google.com/github/LeeSeungwon89/Python_for_Data_Analytics_Science/blob/main/4.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 데이터 전처리하기**

데이터를 분석하기 전에 데이터 품질을 제고하는 데이터 전처리(preprocessing) 작업을 선행해야 합니다. 데이터 분석 결과가 왜곡되는 상황을 방지하고 데이터 분석 정확도를 제고하기 위한 필수 작업입니다. 

데이터 분석 강의, 인터넷 정보, 서적, **파이썬 머신러닝 판다스 데이터 분석(오승환, 정보문화사)**을 참고했습니다.

## **1.1. 누락된 데이터**

특정 데이터를 실수로 입력하지 않거나 파일 형식을 변환하면서 데이터가 손상되는 경우가 많습니다. 전처리 작업을 거치지 않으면 정확한 데이터 분석 결과를 얻기 어렵습니다. 

### **1.1.1 누락된 데이터 확인하기**

데이터 분석을 위한 라이브러리 목록입니다. 복사해서 사용합니다.

In [15]:
# 기본적인 라이브러리입니다.
import time
import random
import math

# 데이터 분석을 위한 라이브러리입니다.
import numpy as np
import pandas as pd

# 수학 라이브러리입니다.
import scipy as sp
import statsmodels.api as sm

# 웹 스크레이핑을 위한 라이브러리입니다
import re
import requests
from bs4 import BeautifulSoup
import os
import json

# 시각화 라이브러리입니다.
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

# 시각화 자료를 바로 띄워줍니다.
%matplotlib inline

# 그래프에 retina를 지정합니다.
%config InlineBackend.figure_format = 'retina'

# 음수 부호를 깨지지 않도록 합니다.
mpl.rc('axes', unicode_minus=False)

사용할 데이터셋은 `seaborn` 라이브러리의 타이타닉 탑승객 정보 데이터셋입니다.

In [16]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [41]:
df.describe

<bound method NDFrame.describe of      survived  pclass     sex   age  ...  deck  embark_town  alive  alone
0           0       3    male  22.0  ...   NaN  Southampton     no  False
1           1       1  female  38.0  ...     C    Cherbourg    yes  False
2           1       3  female  26.0  ...   NaN  Southampton    yes   True
3           1       1  female  35.0  ...     C  Southampton    yes  False
4           0       3    male  35.0  ...   NaN  Southampton     no   True
..        ...     ...     ...   ...  ...   ...          ...    ...    ...
886         0       2    male  27.0  ...   NaN  Southampton     no   True
887         1       1  female  19.0  ...     B  Southampton    yes   True
888         0       3  female   NaN  ...   NaN  Southampton     no  False
889         1       1    male  26.0  ...     C    Cherbourg    yes   True
890         0       3    male  32.0  ...   NaN   Queenstown     no   True

[891 rows x 15 columns]>

`info()` 메서드만으로도 누락된 데이터를 어느 정도 파악할 수 있습니다. 총 데이터 개수는 891인데 'age', 'embarked', 'deck', 'embark_town' 열은 데이터 수가 891 미만으로 모자랍니다. 

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


`isnull()` 메서드를 사용하면 `NaN`을 의미하는 `True`가 1, `False`가 0입니다. 이 값을 열 기준으로 모두 더하는 `sum(axis=0)` 메서드를 활용하면 데이터가 누락된 열 개수를 파악할 수 있습니다. `axis=0`과 `axis=1`은 혼동하기 쉬운 개념입니다. 명확하게 정리하겠습니다.

- `axis=0`: 행(인덱스) 방향으로 작동합니다. 책을 위로 쌓으면서 정리한다고 볼 수 있습니다. 다시 말하면 각 열의 모든 행으로 작동합니다. 작업 결과는 행으로 출력됩니다. 

- `axis=1`: 열(컬럼) 방향으로 작동합니다. 책을 옆으로 정리한다고 볼 수 있습니다. 다시 말하면 각 행의 모든 열로 작동합니다. 작업 결과는 열로 출력됩니다. 

In [18]:
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [43]:
df.isnull().sum(axis=0)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

`notnull()` 메서드는 `isnull()` 메서드의 반대 방향으로 작동합니다.

In [20]:
df.head().notnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [21]:
df.notnull().sum(axis=0)

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

`value_counts()` 메서드는 각 열의 데이터 값마다 개수로 출력합니다. `dropna=False`로 지정하면 `NaN`의 개수도 함께 출력합니다.

In [22]:
df['age'].value_counts(dropna=False)

NaN      177
24.00     30
22.00     27
18.00     26
28.00     25
        ... 
36.50      1
55.50      1
66.00      1
23.50      1
0.42       1
Name: age, Length: 89, dtype: int64

In [23]:
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

반목문으로도 `NaN`을 확인할 수 있습니다. 예외 처리 구문은 [링크](https://wikidocs.net/30)를 참조하시기 바랍니다.

In [31]:
df_nan = df.isnull()

for column in df_nan:
    count_nan = df_nan[column].value_counts()
    try:
        print(f'{column}: {count_nan[True]}')
    # `missing_count`의 데이터에 `True`가 없으면 예외로 처리합니다.
    except:
        print(f'{column}: 0')

survived: 0
pclass: 0
sex: 0
age: 177
sibsp: 0
parch: 0
fare: 0
embarked: 2
class: 0
who: 0
adult_male: 0
deck: 688
embark_town: 2
alive: 0
alone: 0


### **1.1.2. 누락된 데이터 제거하기**

누락된 데이터를 제거하려면 행(레코드)을 삭제하거나 열(특성)을 삭제하는 방법을 취할 수 있습니다. 'deck' 열은 누락된 데이터가 688개이므로 `dropna()` 메서드를 사용하여 열을 통으로 삭제해야 합니다. `axis=0`으로 지정하여 행을 기준으로 삼아 아래로 카운트하고, `thresh=300`으로 지정하여 `NaN`이 300개 이상 카운트된 열을 삭제하겠습니다. 아울러기존 `df` 인스턴스에 변형이 일어나지 않도록 새로운 인스턴스인 `df_thresh`를 생성하겠습니다.

In [46]:
df_thresh = df.dropna(axis=0, thresh=300)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


### **1.1.3. 누락된 데이터 치환하기**

## **1.2. 중복된 데이터**

### **1.2.1. 중복된 데이터 확인하기**

### **1.2.2. 중복된 데이터 제거하기**

## **1.3. 데이터 표준화**

### **1.3.1. 단위 환산하기**

### **1.3.2. 자료형 변환하기**

## **1.4. 범주형 데이터**

### **1.4.1. 구간 분할하기**

### **1.4.2. 더미 변수 사용하기**

## **1.5. 데이터 정규화**

## **1.6. 시계열 데이터**

### **1.6.1. 시계열 객체로 변환하기**

### **1.6.2. 시계열 데이터 생성하기**

### **1.6.3. 시계열 데이터 활용하기**